# Multi-Contingency Transient Stability

Demonstrates batch transient stability simulation with multiple contingency
scenarios and comparative visualization. Several fault types are defined,
solved in sequence, and their dynamic responses compared side-by-side.

In [ ]:
import matplotlib.pyplot as plt
from esapp import PowerWorld, TS
from esapp.components import Bus, Gen
from examples.map import format_plot

In [ ]:
# This cell is hidden in the documentation.
import ast

with open('../data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

pw = PowerWorld(case_path)

In [ ]:
# Plotting functions (hidden from documentation)
import sys; sys.path.insert(0, "..")
from plot_helpers import plot_comparative_dynamics

## 1. Define Multiple Contingencies

Create several contingency scenarios to compare their dynamic impact.

In [ ]:
pw.dyn.runtime = 10.0
pw.dyn.watch(Gen, [TS.Gen.P, TS.Gen.W, TS.Gen.Delta])
pw.dyn.watch(Bus, [TS.Bus.VPU])

# Contingency 1: Bus fault
(pw.dyn.contingency("Bus_Fault")
       .at(1.0).fault_bus("1")
       .at(1.153).clear_fault("1"))

# Contingency 2: Generator trip
(pw.dyn.contingency("Gen_Trip")
       .at(1.0).trip_gen("2", "1"))

# Contingency 3: Branch trip
branches = pw[Gen].head()
(pw.dyn.contingency("Branch_Trip")
       .at(1.0).trip_branch("1", "2", "1"))

## 2. Batch Simulation

Solve all contingencies and collect results.

In [ ]:
ctg_names = ["Bus_Fault", "Gen_Trip", "Branch_Trip"]
all_meta = {}
all_results = {}

for name in ctg_names:
    meta, results = pw.dyn.solve(name)
    all_meta[name] = meta
    all_results[name] = results
    print(f"Solved '{name}': {results.shape[0]} time steps, {results.shape[1]} channels")

In [ ]:
plot_comparative_dynamics(ctg_names, all_results)

## Summary

Batch simulation enables systematic screening of contingencies by solving
each scenario in sequence and collecting results. The comparative plot
reveals which fault types produce the most severe dynamic response,
supporting contingency ranking and worst-case identification.